In [ ]:
import uproot
import pandas as pd
import numpy as np
from os import listdir
import os
import h5py
import awkward
import time

# Metadata

The follwoing file (*infofile.py*) loads a python dictionary (*info*) which stores all the cross-sections and number of events needed to scale each MC process to the right luminosity. The scale factor is given by

$$sf_{\mathscr{L}} = \frac{\sigma[fb]*\mathscr{L}}{N_{ev}^{simulated}}.$$

$N_{ev}^{simulated}$ is the number of originally simulated events for a given MC sample, $\sigma$ is the cross-section in femtobarns [fb] and $\mathscr{L}$ is the integrated luminosity of the data sample. For the 13TeV openData release $\mathscr{L} = 10.6~fb^{-1}$. Note that the cross-section in the infofile is in picobarn [pb] 

In [ ]:
from infofile import infos
info = {} 
for key in infos.keys(): 
    ID = infos[key]['DSID']
    info[ID] = {} 
    info[ID]['xsec'] = infos[key]['xsec'] 
    info[ID]['sumw'] = infos[key]['sumw'] 
    info[ID]['events'] = infos[key]['events']

The lists below define the DSIDs to be identified with each background type.

In [ ]:
SUSYC1C1 = [392501,392502,392504,392506,392507,392509,392513,392517,392518,392521]

SUSYslsl = [392916,392918,392920,392924,392925,392936,392942,392951,392962,392964,392982,392985,392996,392999]

ZPrime_chi = [301322,301323,301324,301325,301326,301327,301328,301329,301330,301331,301332,301333]

Zjets = [364100, 364101, 364102, 364103, 364104, 364105, 364106, 364107, 364108, 364109, 364110, 
         364111, 364112, 364113, 364114, 364115, 364116, 364117, 364118, 364119, 364120, 364121, 
         364122, 364123, 364124, 364125, 364126, 364127, 364128, 364129, 364130, 364131, 364132, 
         364133, 364134, 364135, 364136, 364137, 364138, 364139, 364140, 364141]

Wjets = [364156, 364157, 364158, 364159, 364160, 364161, 364162, 364163, 364164, 364165, 364166, 
         364167, 364168, 364169, 364170, 364171, 364172, 364173, 364174, 364175, 364176, 364177, 
         364178, 364179, 364180, 364181, 364182, 364183, 364184, 364185, 364186, 364187, 364188, 
         364189, 364190, 364191, 364192, 364193, 364194, 364195, 364196, 364197]

Diboson = [361600, 361601, 361602, 361603, 361604, 361606, 361607, 361609, 361610] 

Top = [410000, 410011, 410012, 4100013, 410014, 410025, 410026]

Higgs = [341081, 343981, 345041, 345318, 345319]

fileIDs_bkg = {'Diboson':Diboson, 'Zjets':Zjets, 'Wjets':Wjets, 'Top':Top, 'Higgs':Higgs}
fileIDs_sig = {'SUSYC1C1':SUSYC1C1, 'SUSYslsl':SUSYslsl, 'ZPrime_chi':ZPrime_chi}

This function calculates the scale factor needed for the scaling of MC to the right luminosity. In stead of storing all the individual scale factors, weights and cross-sections this function calculates the final scale factor so that the individual branches (i.e. scaleFactor_*, mc_weight, etc.) do not need to be stored.

In [ ]:
def calc_sf(xsec,lumi,nev,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_BTAG,scaleFactor_LepTRIGGER):
    if lumi <= 0: 
        print("Lumi {:d} is not valid".format(lumi)) 
        return 0
    wgt = (mcWeight)*(scaleFactor_PILEUP)*(scaleFactor_ELE)*(scaleFactor_MUON)*(scaleFactor_BTAG)*(scaleFactor_LepTRIGGER)
    return wgt * ((xsec*1000*lumi)/nev)

The follwoing function is not really used anymore, but maybe it will become useful at some point. So saves it here :-)

In [ ]:
def sortAndIndex(df,nlep):
    # Removes the subentry index (will be replaced by a sorted (in terms of lepton pt) index)
    df.reset_index(level=1, drop=True, inplace=True)
    df = df.set_index(['lep_pt'],append=True).sort_index(ascending=False)
    if (df.shape[0] % nlep) != 0:
        print("WARNING\t Shape is odd {:d}".format(df.shape[0]))
        size  = int(df.shape[0]/nlep)+1
    else:
        size  = int(df.shape[0]/nlep)
    serie = [x for x in range(1,nlep+1)]
    s = pd.Series(serie*size)
    if s.shape[0] != df.shape[0]:
        print("ERROR \t Shape mismatch serie: {:d} vs. data frame: {:d}".format(s.shape[0],df.shape[0]))
        print(s)
    df.set_index([s],append=True,inplace=True)
    df.reset_index(level=1,inplace=True)
    df.rename_axis(('entry','lepnum'),inplace=True)
    return df

# Skimming
The following function specify the skimming. Due to memory consumption it is not feasible to read all events so some skimming is required

In [ ]:
def skimming(df,events,prev_n,next_n,nlep,lep_ptcut):
    #pt, met = events.arrays(['lep_pt','met_et'],outputtype=tuple,entrystart=prev_n,entrystop=next_n)
    # Vector to hold all the 
    
    pt  = awkward.fromiter(df['lep_pt'])
    met = awkward.fromiter(df['met_et'])
    
    skim = []
    
    # MET > 50 GeV
    skim.append(pd.Series(met > 50000,name='bools').astype(int))
    
    # Makes sure that each sub-vector has the same number of entries (filling -999 if not)
    pt = pt.pad(nlep).fillna(-999)
    
    # Require leptons to have enough pt (according to values set in lep_ptcut)
    for i in range(len(lep_ptcut)):
        se = pt > lep_ptcut[i]
        skim.append(pd.Series(pt[pt > lep_ptcut[i]].counts >= 1).astype(int))
        mask = np.logical_and(pt != pt.max(), pt.max != -999)
        pt = pt[mask]
        
     
    # Make sure we only have exactly nlep (after the pt cuts)
    skim.append(pd.Series(pt[pt > 0].counts == 0).astype(int))
    
        
    # < here one can add additional cuts. Remeber to append the result to the skim vector)
    
    # Make sure that all our entries in the skim vector has value 1 
    # If not this means that one of the cuts above did not pass (i.e. we don't want to keep the event)
    # Adding values from all skim vectors together should give a total equal to the length of the skim vector
    sk_final = skim[0]
    for i in range(1,len(skim)):
        sk_final = sk_final.add(skim[i])
    final_skim = pd.Series(sk_final == len(skim))
    
    # Keep only rows where we have right number of leptons with pT above thresholds
    df = df[final_skim.values]
    
    return df

# Augmentation
The following function let you add new jet variables into the panda data frame. The jet information is removed apriori as the exact number of jets is varying from event to event. 

In [ ]:
def jetaugmentation(df,events,prev_n,next_n):
    #pt, eta, phi, e = events.arrays(['jet_pt','jet_eta','jet_phi','jet_E'],outputtype=tuple,entrystart=prev_n,entrystop=next_n)
    pt  = awkward.fromiter(df['jet_pt'])
    eta = awkward.fromiter(df['jet_eta'])
    phi = awkward.fromiter(df['jet_phi'])
    e   = awkward.fromiter(df['jet_E'])
    df['jet_n60'] = pt[pt > 60000].counts
    return df

The following function let you add lepton variables. The jagged arrays need to be turned into variables for each lepton. If you apply skimming on 2 leptons the information of the two hardest leptons will be saved. If skimming is set to 3 it saves the 3 hardest leptons etc. One can also add higher level variables like mll, deltaR etc. See documentation here: [here](https://github.com/scikit-hep/uproot#multiple-values-per-event-jagged-arrays)

In [ ]:
def lepaugmentation(df,events,prev_n,next_n,nlep):
    #pt, eta, phi, e = events.arrays(['lep_pt','lep_eta','lep_phi','lep_E'],outputtype=tuple,entrystart=prev_n,entrystop=next_n)
    
    #print(df.keys())
    #subset = df[['lep_pt']]#,'lep_eta','lep_phi','lep_E']]
    #pt = tuple(subset.values)
    #pt = fromparents(df['lep_pt'])
    pt  = awkward.fromiter(df['lep_pt'])
    eta = awkward.fromiter(df['lep_eta'])
    phi = awkward.fromiter(df['lep_phi'])
    e   = awkward.fromiter(df['lep_E'])
    
    # Compute mll for all combinations of two-lepton pairs
    pairs = pt.argchoose(2)
    left  = pairs.i0
    right = pairs.i1
    masses = np.sqrt(2*pt[left]*pt[right]*(np.cosh(eta[left]-eta[right])-np.cos(phi[left]-phi[right])))
    df['mll'] = masses.content
    
    pt = pt.pad(nlep).fillna(-999)
    eta = eta.pad(nlep).fillna(-999)
    phi = phi.pad(nlep).fillna(-999)
    e = e.pad(nlep).fillna(-999)

    # Make the lepton variables
    for i in range(1,nlep+1):
        df['lep%i_pt'%i]  = pt[pt.argmax(),:].flatten()
        df['lep%i_eta'%i] = eta[pt.argmax(),:].flatten()
        df['lep%i_phi'%i] = phi[pt.argmax(),:].flatten()
        df['lep%i_E'%i]   = e[pt.argmax(),:].flatten()
        # Remove the hardest and continue to find the next to hardest etc.
        mask = np.logical_and(pt != pt.max(), pt.max != -999)
        pt  = pt[mask]
        eta = eta[mask]
        phi = phi[mask]
        e   = e[mask]
    return df

In [ ]:
def OpenDataPandaFramework(indir,nlep,chunksize,printevry,datatype,skimtag,isMC,isData,isSignal,lep_ptcut,branches,lumi):
    # Count how many events and files we write/read
    n_allfiles   = 0
    nfile   = 0
    totev   = 0
    totev_skim = 0
    out_filenum = 1
    try:
        del [result]
    except:
        print("WARNING\t Result does not exists. Good :-)")
    root_files = [f for f in listdir(indir) if f.endswith('.root')]

    # Checking if file exitst (not needed)
    #path = indir+"/%s_%s.h5" %(datatype,skimtag)
    #if os.path.exists(path):
    #    os.remove(path)
    #    print("WARNING\t Removing file {:s}".format(path))

    for f in root_files:
        #if not "410012" in f: continue
        n_allfiles += 1
        # Getting the file and extracting the information from info dictionary
        events = uproot.open(indir+"/"+f)["mini"]
        nentries = events.numentries
        print("INFO  \t Opening file {:d}/{:d}: {:s} with {:d} events".format(n_allfiles,len(root_files),f,nentries))
        path = indir+"/%s_%s.h5" %(f,skimtag)
        if not isData:
            file_id = int(f.split('.')[1])
            if not file_id in info.keys():
                print("ERROR \t Could not find any info for file id {:d}. Skipping.".format(file_id))
                continue
            xsec = float(info[file_id]['xsec'])
            nev  = float(info[file_id]['sumw'])


            # Find the MC type (defined by dictonaries a few cells above)
            mccat = "" 
            if isMC:
                for key in fileIDs_bkg.keys():
                    if file_id in fileIDs_bkg[key]:       
                        mccat = key
                        break
            elif isSignal:
                for key in fileIDs_sig.keys():
                    if file_id in fileIDs_sig[key]:       
                        mccat = key
                        break
            if not mccat:
                print("ERROR \t Could not find category for DSID {:d}. Skipping".format(file_id))
                continue
            print("INFO  \t ID {:d} in category {:s} has xsec = {:.1f} fb and nev = {:.2f} ".format(file_id,mccat,xsec,nev))


        n = 1
        prev_n = 0

        while True:

            # Measure time to read 
            if n == 1: start = time.time()   
            else:   
                end = time.time()
                dur = (end - start)
                dur_sec = chunksize/dur
                m, s = divmod((nentries-((n-1)*chunksize))/dur_sec, 60)
                h, m = divmod(m, 60)
                print("INFO  \t Event/sec  = {:.0f}. ETC = {:d}h{:02d}m{:02d}s".format(dur_sec,int(h),int(m),int(s)))
                start = time.time()

            # Get the range of events to read
            next_n = n*chunksize if n*chunksize < nentries else nentries
            print("INFO  \t Reading entries {:d} - {:d} of {:d}. Total so far: {:d}".format(prev_n,next_n,nentries,totev_skim))
            df = events.pandas.df(branches,flatten=False,entrystart=prev_n,entrystop=next_n)

            #print(df.keys)

            totev += len(df.index)

            # If MC: get the scale factor corresponding to the total data luminosity (is set above). 
            # If data: set scale factor to 1!
            if not "data" in f:
                df['wgt'] = np.vectorize(calc_sf)(xsec,lumi,nev,df.mcWeight,
                                                  df.scaleFactor_PILEUP,df.scaleFactor_ELE,
                                                  df.scaleFactor_MUON,df.scaleFactor_BTAG,
                                                  df.scaleFactor_LepTRIGGER)
            else:
                df['wgt'] = 1.0

            # Define if it is signal or background (important for ML classification)
            if isMC or isData:
                df['isSignal'] = 0
            elif isSignal:
                df['isSignal'] = 1
            else:
                df['isSignal'] = 0

            if not isData: df['MCType'] = mccat
            else: df['MCType'] = "Data"

            ###########   
            # Skimming
            ###########  
            df = skimming(df,events,prev_n,next_n,nlep,lep_ptcut)

            ##############
            # Augumenting
            ##############
            df = jetaugmentation(df,events,prev_n,next_n)
            df = lepaugmentation(df,events,prev_n,next_n,nlep)

            ###########   
            # Slimming
            ###########
            df = df.drop(['mcWeight','scaleFactor_PILEUP','scaleFactor_ELE','scaleFactor_MUON','scaleFactor_BTAG','scaleFactor_LepTRIGGER'],axis=1)
            df = df.drop(['jet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_jvt'],axis=1)
            df = df.drop(['jet_trueflav', 'jet_truthMatched', 'jet_MV2c10', 'jet_pt_syst'],axis=1)
            df = df.drop(['lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_pt', 'lep_eta'],axis=1)
            df = df.drop(['lep_phi', 'lep_E', 'lep_z0', 'lep_charge', 'lep_type', 'lep_isTightID'],axis=1)
            df = df.drop(['lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased'],axis=1)
            df = df.drop(['lep_tracksigd0pvunbiased', 'lep_pt_syst'],axis=1)
            # If first time, create result data frame. If not; concatenate
            try: 
                result = pd.concat([result,df])
            except:
                result = df
                print("WARNING\t Starting a new result panda")


            totev_skim += len(df.index)

            # Delete the temporary data frame
            del [df]

            if totev_skim > printevry:
                #result = sortAndIndex(result,nlep)
                path = indir+"/%s_%s_num_%i.h5" %(datatype,skimtag,out_filenum)
                result.to_hdf(path,'mini',mode='w',table=True)
                print("INFO  \t Read {:d} events in {:d} files, for which {:d} ({:.2f}%) were written to {:s}"
                    .format(totev,nfile,totev_skim,(float(totev_skim)/float(totev))*100.,path))
                out_filenum += 1
                totev = 0
                totev_skim = 0
                nfile = 0
                del [result]

            # If read everything
            if n*chunksize > nentries: break

            # Update counters before continuing
            prev_n = n*chunksize + 1
            n += 1
            #break

        nfile += 1
        del [events]  
        #break
    # Make sure we write the last events
    #result = sortAndIndex(result,nlep)
    path = indir+"/%s_%s_num_%i.h5" %(datatype,skimtag,out_filenum)
    result.to_hdf(path,'mini',mode='w',table=True)
    print("INFO  \t Read {:d} events in {:d} files, for which {:d} ({:.2f}%) were written to {:s}"
        .format(totev,nfile,totev_skim,(float(totev_skim)/float(totev))*100.,path))
    #del [result]
    return result